# Project Four

* The goal of this project was compare linear regression analysis with basic machine learning algorithms such as LASSO & Ridge.


Data used in project 4 are:

* Monthly patterns data merged with core places data:
  - Under `Grace` workspace, `public/clean-data`.
  - Monthly patterns from 4 months: Jan, Apr, Jul, Oct are used.
  - An additional variable, `poi_cbg`, which is also called *FIPS code* is included. This is the identifier for Census data.
  - The data is cleaned a little. For example splitting 'visits_by_day' into dailyvisits, and transform the data from wide to long.
  
* Census data
  - The data used is under `Grace` workspace, `public/clean-data/cbg.csv`.
  - Complete data under `Grace` workspace, `public/safegraph_open_census_data/data`.
  - Data description: https://www.safegraph.com/blog/beginners-guide-to-census
  - Ziqiao (2021) stated, "it consists of multiple datasets, each representing survey results of a category. 
    + 01 Age; Sex
    + 02 Race
    + 03 Hispanic or Latino Origin
    + 07 Migration/Residence 1 Year Ago
    + 08 Commuting (Journey to Work); Place of Work
    + 09 Relationship to Householder
    + 11 Household Type; Family Type; Subfamilies
    + 12 Marital Status; Marital History
    + 17 Poverty Status
    + 19 Income
    + 20 Earnings
    + 21 Veteran Status; Period of Military Service
    + 22 Food Stamps/Supplemental Nutrition Assistance Program (SNAP)
    + 23 Employment Status; Work Status Last Year 
    + 24 Industry, Occupation, and Class of Worker
    
    i.e. data `cbg_b01.csv` contains survey results on population age and sex, each row identified by `census_block_group`, which is the *FIPS code*, and is the same as the variable `poi_cbg` in the patterns data above" (Ziqiao, Elms)
  - The variable names consist of letters and numbers. A complete description of what each column refers to is in the file `public/safegraph_open_census_data/metadata/cbg_field_descriptions.csv`, which can be open using excel.
  
  
* US covid cases and death
  - Under `Grace` workspace, `public/clean-data/us-counties.csv`.
  - The data is arranged by county.
  - Safegraph data is by city. To match the US covid data with Safegraph data, the FIPS code provided is used.
* For description of other data we are using, refer to the project.

# Part 1: Prepare Data

Question 1a stated, "in GRACE, go to `public/clean-data`, download the four datasets of Safegraph dailyvisits for January, April, July, and October 2020 respectively. From each of them, extract a subsample for your city. Concatenate the four subsamples into one, and reshape it into the long form as you have done in Project 1. Each row of your long data should represent a date and a Safegraph place id."

In [1]:
# Modules are imported below:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
noaa = pd.read_csv('noaa.csv')

In [3]:
# # define bizgroups
# df.loc[df['naics_code'] == '722511', 'bizgroup'] = '1' # full service restaurant
# df.loc[df['naics_code'].str.startswith('721'), 'bizgroup'] = '2' # accomodation
# df.loc[df['naics_code'].str.startswith('4451'), 'bizgroup'] = '3' # grocery
# df.loc[df['naics_code'] == '92', 'bizgroup'] = '4' # Public Administration
# mlong.to_csv('mlong.csv', index = False)

* The file mlong.csv is the long form of the four datasets of Safegraph dailyvisits for January, April, July, and October 2020 respectively that were download. the file mlong.csv below is the final result from extracting a subsample from my city Cheyenne. Mlong is the also the final product from concatenating the four subsamples into one, and reshape it into the long form as done in Project 1. Each row of mlong data represents a date and a Safegraph place id." 

In [4]:
mlong = pd.read_csv('mlong.csv')

In [5]:
mlong.columns = ['id', 'dayofmonth', 'poi_cbg', 'region', 'naics_code', 'safegraph_place_id', 'date_range_start', 'city', 'bizgroup', 'dailyvisits', 'date_range_end', 'dayofweek', 'dayname', 'weekend']

In [6]:
mlong.head()
mlong.tail()

,id,dayofmonth,poi_cbg,region,naics_code,safegraph_place_id,date_range_start,city,bizgroup,dailyvisits,date_range_end,dayofweek,dayname,weekend
12812,2296696,30,560210003002,WY,445120,sg:10580842008346c081f663e0a86130f0,2020-10-01,Cheyenne,3.0,11,2020-10-31,5,Sat,1
12813,2301355,30,560210014022,WY,722511,sg:cbd27e353c2c46bba9bedc85e29f1003,2020-10-01,Cheyenne,1.0,4,2020-10-31,5,Sat,1
12814,2302531,30,560210014022,WY,446110,sg:4aad029e90cb4268a3b8901db548a73e,2020-10-01,Cheyenne,NaN,3,2020-10-31,5,Sat,1
12815,2302572,30,560210014022,WY,722511,sg:66018d6d8b0d4a3bbbe000b60a29ba4f,2020-10-01,Cheyenne,1.0,38,2020-10-31,5,Sat,1
12816,2311094,30,560210012003,WY,446110,sg:efae6f525560496da71126d9acd2cf68,2020-10-01,Cheyenne,NaN,15,2020-10-31,5,Sat,1


The answer to question 1a is the file mlong.csv above is the reshaped long sample. It has a subsample from the city Cheyenne. Each row of the long data represent a date and a Safegraph place id.

Question 1b asked,

* How many observations do you have in your “LocalSafegraph”? 
* How many unique places are there? 
* How many unique NAICS codes are there? 
* What type of places do these NAICS codes represent? 
* How many unique places do you have for each NACIS code? 
* What is the average population for each observation in your “LocalSafeGraph” data?
* How many unique values of `FIPS_5digits’ do you have in “LocalSafegraph”? Ideally, you should have only one value of `FIPS_5digits’, which represents the state-county your city locates in. 


### Census data

Below is the census data used. They are concat together and named `cbgg.csv` data.

Below is description of the variables.

* `cbgg.csv`:
  - `B01001e1`, `B01001e2` are total population, and total male population.
  - `B01002e1`, `B01002e2`, `B01002e3` are median age (for total population, for male, and for female, respectively).
  - `B02001e2`, `B02001e3`, `B02001e5` are total white, black & asian population.
  - `B11001e1-3` are total number of Household, # of family HH, and # of family HH with married couples, respectively.
  - `B15003e1` is population of 25 yrs and older.
  - `B15003e2-18` are population of high school and below.
  - `B15003e19-25` are population of college and higher.
  - `B19013e1`, `B19301e1`, `B19025e1` are median HH income, per capita income, aggregate HH income, respectively.

In [7]:
# Census data is downloaded below, including the match id 'census_block_group' and 16 variables:
cbgg = pd.read_csv('cbgg.csv',
                 dtype = {'census_block_group': 'object'})
cbgg.head()

,census_block_group,B01001e1,B01001e2,B01002e1,B01002e2,B01002e3,B02001e2,B02001e3,B02001e5,B11001e1,B11001e2,B11001e3,B11001e4,B11001e5,B11001e6,B11001e7,B11001e8,B11001e9,B15003e1,B15003e2,B15003e3,B15003e4,B15003e5,B15003e6,B15003e7,B15003e8,B15003e9,B15003e10,B15003e11,B15003e12,B15003e13,B15003e14,B15003e15,B15003e16,B15003e17,B15003e18,B15003e19,B15003e20,B15003e21,B15003e22,B15003e23,B15003e24,B15003e25,B19013e1,B19025e1,B19301e1
0,010010201001,745,356,34.1,37.1,33.7,585,160,0,284,193,112,81,11,70,91,63,28,474,0,0,0,0,0,0,0,0,23,0,0,53,6,10,11,128,21,13,88,15,60,23,7,16,NaN,14126600.0,20365.0
1,010010201002,1265,639,41.8,39.9,44.8,1083,104,9,456,390,310,80,4,76,66,58,8,824,7,0,0,0,0,0,0,0,0,0,0,27,10,11,9,216,51,21,137,34,156,118,18,9,77813.0,41514300.0,33336.0
2,010010202001,960,534,38.2,23.6,45.0,361,568,0,386,244,145,99,6,93,142,133,9,602,9,0,0,0,0,0,4,15,0,0,6,17,4,63,3,211,51,70,64,15,54,7,9,0,25179.0,15608200.0,17047.0
3,010010202002,1236,634,39.7,33.5,43.3,615,571,24,452,286,184,102,9,93,166,153,13,861,7,0,0,0,0,0,0,0,6,6,16,18,48,28,24,255,59,63,90,74,113,39,9,6,45104.0,25847500.0,21400.0
4,010010203001,2364,1125,34.9,34.1,39.3,1481,515,27,824,560,435,125,0,125,264,228,36,1648,46,0,0,18,0,0,0,0,0,5,5,39,42,33,20,536,40,122,264,112,205,133,10,18,55222.0,50827900.0,23106.0


Below, several variables that may help with predicting dailyvisits are included. Two variables that B15003e2-18: high school and B15003e19-25: college and higher are created and added to cbgg file.

In [8]:

cbgg['EDU_under'] = cbgg.loc[:, 'B15003e2':'B15003e16'].sum(axis = 1) / cbgg['B15003e1']
cbgg['EDU_higher'] = cbgg.loc[:, 'B15003e21':'B15003e25'].sum(axis = 1) / cbgg['B15003e1']

cbg1 = cbgg.loc[:, ['census_block_group','B01001e1', 'B01001e2', 'B01002e1', 
               'B01002e2','B01002e3', 'B02001e2', 'B02001e3', 'B02001e5', 
               'B11001e1', 'B11001e2', 'B11001e3', 
               'EDU_under', 'EDU_higher', 'B19013e1', 'B19025e1', 'B19301e1']]
cbg1.head()

,census_block_group,B01001e1,B01001e2,B01002e1,B01002e2,B01002e3,B02001e2,B02001e3,B02001e5,B11001e1,B11001e2,B11001e3,EDU_under,EDU_higher,B19013e1,B19025e1,B19301e1
0,010010201001,745,356,34.1,37.1,33.7,585,160,0,284,193,112,0.217300,0.255274,NaN,14126600.0,20365.0
1,010010201002,1265,639,41.8,39.9,44.8,1083,104,9,456,390,310,0.077670,0.406553,77813.0,41514300.0,33336.0
2,010010202001,960,534,38.2,23.6,45.0,361,568,0,386,244,145,0.200997,0.141196,25179.0,15608200.0,17047.0
3,010010202002,1236,634,39.7,33.5,43.3,615,571,24,452,286,184,0.177700,0.279907,45104.0,25847500.0,21400.0
4,010010203001,2364,1125,34.9,34.1,39.3,1481,515,27,824,560,435,0.126214,0.290049,55222.0,50827900.0,23106.0


In [9]:
# Here I rename the columns:
cbg1.columns = ['census_block_group', 'tot_pop', 'tot_male_pop', 
                'median_age_tot_pop', 'median_age_male', 'median_age_female', 'tot_white_pop', 'tot_blk_pop', 'tot_Asian_pop', 
                'tot_num_house-holds', 'num_of_family_HH', 'num_of_fam_HH_with_marr_couples', 'EDU_under', 'EDU_higher', 'median_HH_income', 
                'aggregate_HH_income', 'per_capita_income']
cbg1.head()

,census_block_group,tot_pop,tot_male_pop,median_age_tot_pop,median_age_male,median_age_female,tot_white_pop,tot_blk_pop,tot_Asian_pop,tot_num_house-holds,num_of_family_HH,num_of_fam_HH_with_marr_couples,EDU_under,EDU_higher,median_HH_income,aggregate_HH_income,per_capita_income
0,010010201001,745,356,34.1,37.1,33.7,585,160,0,284,193,112,0.217300,0.255274,NaN,14126600.0,20365.0
1,010010201002,1265,639,41.8,39.9,44.8,1083,104,9,456,390,310,0.077670,0.406553,77813.0,41514300.0,33336.0
2,010010202001,960,534,38.2,23.6,45.0,361,568,0,386,244,145,0.200997,0.141196,25179.0,15608200.0,17047.0
3,010010202002,1236,634,39.7,33.5,43.3,615,571,24,452,286,184,0.177700,0.279907,45104.0,25847500.0,21400.0
4,010010203001,2364,1125,34.9,34.1,39.3,1481,515,27,824,560,435,0.126214,0.290049,55222.0,50827900.0,23106.0


In [10]:
# Long data is renamed below:
df = pd.read_csv('mlong.csv',
                dtype = {'poi_cbg':'object',
                          'naics_code': 'object'})
df.columns = ['id', 'dayofmonth', 'poi_cbg', 'region', 'naics_code', 'safegraph_place_id', 'date_range_start', 'city',
                 'bizgroup', 'dailyvisits', 'date_range_end', 'dayofweek', 'dayname', 'weekend']
df.head()

,id,dayofmonth,poi_cbg,region,naics_code,safegraph_place_id,date_range_start,city,bizgroup,dailyvisits,date_range_end,dayofweek,dayname,weekend
0,277,0,560210014022,WY,722511,sg:9d80429296d44fc0b570ba35884e3574,2020-01-01,Cheyenne,1.0,29,2020-01-01,2,Wed,0
1,3594,0,560210007001,WY,722511,sg:0ccd14ce55944dc99bc847781b323da0,2020-01-01,Cheyenne,1.0,3,2020-01-01,2,Wed,0
2,11671,0,560210006001,WY,445110,sg:5bcfc84e23dd48bebd793e73385a7e59,2020-01-01,Cheyenne,3.0,2,2020-01-01,2,Wed,0
3,12917,0,560210004022,WY,445120,sg:a829beef90cc4ef193f77ef8b3bafe9e,2020-01-01,Cheyenne,3.0,0,2020-01-01,2,Wed,0
4,13938,0,560210002002,WY,722511,sg:6a496f98ab8649b0b655de9147159efb,2020-01-01,Cheyenne,1.0,1,2020-01-01,2,Wed,0


Now to get the final data that is used to answer question 1b, census data is merged with the original long data. The new data is called `df2`.

In [11]:
# This is the merged data that is called 'localsafegraph data'
df2 = pd.merge(df, cbg1, left_on = 'poi_cbg', right_on = 'census_block_group', how = 'inner')
df2.drop(columns = 'census_block_group', inplace = True)

In [12]:
df2.head()

,id,dayofmonth,poi_cbg,region,naics_code,safegraph_place_id,date_range_start,city,bizgroup,dailyvisits,date_range_end,dayofweek,dayname,weekend,tot_pop,tot_male_pop,median_age_tot_pop,median_age_male,median_age_female,tot_white_pop,tot_blk_pop,tot_Asian_pop,tot_num_house-holds,num_of_family_HH,num_of_fam_HH_with_marr_couples,EDU_under,EDU_higher,median_HH_income,aggregate_HH_income,per_capita_income
0,277,0,560210014022,WY,722511,sg:9d80429296d44fc0b570ba35884e3574,2020-01-01,Cheyenne,1.0,29,2020-01-01,2,Wed,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0
1,30600,0,560210014022,WY,722511,sg:e8d03235c13149c493506020aa372be5,2020-01-01,Cheyenne,1.0,17,2020-01-01,2,Wed,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0
2,45642,0,560210014022,WY,722511,sg:86decda2bdbe4ed0b1348c4d4cc5ac48,2020-01-01,Cheyenne,1.0,22,2020-01-01,2,Wed,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0
3,91385,0,560210014022,WY,722511,sg:cbd27e353c2c46bba9bedc85e29f1003,2020-01-01,Cheyenne,1.0,10,2020-01-01,2,Wed,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0
4,126930,0,560210014022,WY,445110,sg:502817c99dde4625a568df6ddf84a000,2020-01-01,Cheyenne,3.0,1,2020-01-01,2,Wed,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0


* Missing census FIPS code are not allowed, a check for missing FIPS is done below before proceeding. 

In [13]:
# This code double-checks if all rows in df merged into df2
print(len(df) - len(df2))

0


In [14]:
# This drop all rows with missing values.
# It is run just to make sure missing code are not in the file.
df2 = df2[~df2.isna().any(axis=1)].copy()

In [15]:
# The code below finds how many observations ae in the merged data.
len(df2)

11986

In [16]:
# The code below finds how many unique place ID there are:
df2['safegraph_place_id'].nunique()

114

In [17]:
df2['naics_code'].nunique()

4

In [18]:
df2['safegraph_place_id'].describe()

count                                   11986
unique                                    114
top       sg:69169ad2c1404f968f9f00753d9a8d27
freq                                      123
Name: safegraph_place_id, dtype: object

In [19]:
# The code below shows what each column type is in the dataset:
df2.dtypes

id                                   int64
dayofmonth                           int64
poi_cbg                             object
region                              object
naics_code                          object
safegraph_place_id                  object
date_range_start                    object
city                                object
bizgroup                           float64
dailyvisits                          int64
date_range_end                      object
dayofweek                            int64
dayname                             object
weekend                              int64
tot_pop                              int64
tot_male_pop                         int64
median_age_tot_pop                 float64
median_age_male                    float64
median_age_female                  float64
tot_white_pop                        int64
tot_blk_pop                          int64
tot_Asian_pop                        int64
tot_num_house-holds                  int64
num_of_fami

In [20]:
df2['bizgroup'].nunique()

3

In [21]:
# The code below defines a separate string variable `FIPS_5digits’ as the first five digits of `poi_cbg’. 
df2['FIPS_5digits'] = df2['poi_cbg'].str[0:5]
df2

,id,dayofmonth,poi_cbg,region,naics_code,safegraph_place_id,date_range_start,city,bizgroup,dailyvisits,date_range_end,dayofweek,dayname,weekend,tot_pop,tot_male_pop,median_age_tot_pop,median_age_male,median_age_female,tot_white_pop,tot_blk_pop,tot_Asian_pop,tot_num_house-holds,num_of_family_HH,num_of_fam_HH_with_marr_couples,EDU_under,EDU_higher,median_HH_income,aggregate_HH_income,per_capita_income,FIPS_5digits
0,277,0,560210014022,WY,722511,sg:9d80429296d44fc0b570ba35884e3574,2020-01-01,Cheyenne,1.0,29,2020-01-01,2,Wed,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,56021
1,30600,0,560210014022,WY,722511,sg:e8d03235c13149c493506020aa372be5,2020-01-01,Cheyenne,1.0,17,2020-01-01,2,Wed,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,56021
2,45642,0,560210014022,WY,722511,sg:86decda2bdbe4ed0b1348c4d4cc5ac48,2020-01-01,Cheyenne,1.0,22,2020-01-01,2,Wed,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,56021
3,91385,0,560210014022,WY,722511,sg:cbd27e353c2c46bba9bedc85e29f1003,2020-01-01,Cheyenne,1.0,10,2020-01-01,2,Wed,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,56021
4,126930,0,560210014022,WY,445110,sg:502817c99dde4625a568df6ddf84a000,2020-01-01,Cheyenne,3.0,1,2020-01-01,2,Wed,0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,56021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12812,1981319,26,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,29,2020-10-27,1,Tue,0,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.000000,0.391204,41648.0,16304800.0,25449.0,56021
12813,1981319,27,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,29,2020-10-28,2,Wed,0,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.000000,0.391204,41648.0,16304800.0,25449.0,56021
12814,1981319,28,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,21,2020-10-29,3,Thur,0,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.000000,0.391204,41648.0,16304800.0,25449.0,56021
12815,1981319,29,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,21,2020-10-30,4,Fri,0,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.000000,0.391204,41648.0,16304800.0,25449.0,56021


In [22]:
df2['FIPS_5digits'].nunique()

2

* The answer to how many observations do you have in your “LocalSafegraph”is  11986 observations are in the 'LocalSafeGraph' data. 
* The answer to how many unique places are there is 114 unique place ID are in the dataset.
* The answer to how many unique NAICS codes are there is 4 unique NAICS codes are there.

* The answer to what type of places do these NAICS codes represent is the NAICS codes represent full service restaurant, accomodation, grocery, and public administration.
* The answer to how many unique places do you have for each NACIS code is 4.
* The answer to what is the average population for each observation in your “LocalSafeGraph” data is 123.

* The answer to how many unique values of 'FIPS_5digits' do you have in “LocalSafegraph is 2 unique values of 'FIPS_5digits' are in the 'localsafegraph' data.

Question 1c (1 point): In GRACE, go to `public/clean-data`, download the dataset of COVID-19 cases as collected by NY Times. Extract the relevant observations for your city using the value of FIPS_5digits you have found in Question 1b. I will refer to this data as your “LocalCOVIDcases”.

 

How many observations are there in your “LocalCOVIDcases”? What is the minimum date? What is the maximum date? What is the average number of daily cases? What is the average number of daily deaths?

In [23]:
cc = pd.read_csv('us-counties.csv', dtype={'fips':'object'})
cc.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0.0
1,2020-01-22,Snohomish,Washington,53061,1,0.0
2,2020-01-23,Snohomish,Washington,53061,1,0.0
3,2020-01-24,Cook,Illinois,17031,1,0.0
4,2020-01-24,Snohomish,Washington,53061,1,0.0


In [24]:
# This code gets the new dataset called 'localcovidcases'
cc1 = cc.loc[cc['fips'].isin(['56021']), :]
cc1.sort_values(by = 'date', ascending = True)
cc1.head()

,date,county,state,fips,cases,deaths
3757,2020-03-17,Laramie,Wyoming,56021,2,0.0
4396,2020-03-18,Laramie,Wyoming,56021,3,0.0
5162,2020-03-19,Laramie,Wyoming,56021,4,0.0
6077,2020-03-20,Laramie,Wyoming,56021,4,0.0
7111,2020-03-21,Laramie,Wyoming,56021,5,0.0


In [25]:
cc1.columns = ['date_range_start', 'county', 'state', 'fips', 'cases', 'deaths']

In [26]:
cc1.tail()

,date_range_start,county,state,fips,cases,deaths
1154124,2021-03-24,Laramie,Wyoming,56021,8519,110.0
1157371,2021-03-25,Laramie,Wyoming,56021,8528,110.0
1160619,2021-03-26,Laramie,Wyoming,56021,8561,110.0
1163867,2021-03-27,Laramie,Wyoming,56021,8561,110.0
1167115,2021-03-28,Laramie,Wyoming,56021,8561,110.0


In [27]:
len(cc1)

377

In [28]:
cc1['cases'].describe()

count     377.000000
mean     2902.782493
std      3325.350354
min         2.000000
25%       204.000000
50%       652.000000
75%      6807.000000
max      8561.000000
Name: cases, dtype: float64

In [29]:
cc1['deaths'].describe()

count    377.000000
mean      27.952255
std       38.894126
min        0.000000
25%        2.000000
50%        4.000000
75%       51.000000
max      110.000000
Name: deaths, dtype: float64

Question 1c asked, how many observations are there in your “LocalCOVIDcases”? 377 observations are in the 'LocalCOVIDcases' data. It also asked, what is the minimum date? The minimum date is March 17th, 2020. What is the maximum date? The maximum date is March 28th, 2021. What is the average number of daily cases? The average number of cases was 2,902.7825. What is the average number of daily deaths? The average number of deaths was 27.9523.

Question 1d stated, from the NOAA dataset downloaded to begin the project, how many observations are there in your “LocalWeather”? What is the average air temperature? What is the average precipitation?

In [30]:
noaa.columns = ['STATION', 'NAME', 'date_range_start', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS']
noaa.head()
# PRCP = Precipitation; SNOW = Snow Fall; SNWD = Snow Depth;
# TMAX = Max temperature; TMIN = Min Temperature; 
# TOBS = Temperature at time of Observation.

,STATION,NAME,date_range_start,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS
0,USC00481676,"CHEYENNE WEATHER FORECAST OFFICE, WY US",2020-01-01,0.0,0.0,1.0,40,29,31
1,USC00481676,"CHEYENNE WEATHER FORECAST OFFICE, WY US",2020-01-02,0.0,0.0,0.0,38,22,22
2,USC00481676,"CHEYENNE WEATHER FORECAST OFFICE, WY US",2020-01-03,0.0,0.0,0.0,41,19,33
3,USC00481676,"CHEYENNE WEATHER FORECAST OFFICE, WY US",2020-01-04,0.0,0.0,0.0,55,33,33
4,USC00481676,"CHEYENNE WEATHER FORECAST OFFICE, WY US",2020-01-05,0.0,0.0,0.0,33,24,26


In [31]:
len(noaa)

366

In [32]:
noaa['TOBS'].describe()

count    366.000000
mean      41.060109
std       16.659250
min        1.000000
25%       28.000000
50%       39.500000
75%       57.000000
max       71.000000
Name: TOBS, dtype: float64

In [33]:
noaa['PRCP'].describe()

count    366.000000
mean       0.027213
std        0.086207
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        0.780000
Name: PRCP, dtype: float64

Here are the answers to the questions in 1d, how many observations are there in your “LocalWeather”? 366 observations are in the 'localweather' data. What is the average air temperature? The average temperature is 41.0601. What is the average precipitation? The average precipitation is .0272.

# Part 2: Merge data for final analysis sample

Question 2a (2 points): Now merge your “LocalSafegraph” with “LocalCovidcases” and “LocalWeather” by date. Keep all dates that appear in “LocalSafegraph”, even if they do not appear in “LocalCovidcases” or “LocalWeather”. Do not include dates not showing up in “LocalSafegraph”. I will refer to the merged data as “LocalMerged”.

 

How many dates in your “LocalMerged” have missing values in the daily count of COVID cases and COVID deaths? Impute these missing values as COVID cases equal to 0 and COVID deaths equal to 0. (NY Times only report COVID information after a COVID case started to occur in a local county.) After the imputation, what are the average daily COVID cases and daily COVID deaths in your “LocalMerged”?

 

Do you have any date that has missing values in air temperature or precipitation? If so, drop these dates from “LocalMerged”.  How many observations do you have now in “LocalMerged”?

In [34]:
# Here df2 = 'LocalSafegraph,' cc1 = 'LocalCovidcases,' 
# and noaa = 'LocalWeather' are merged and is called 'LocalMerged(df4)'
df3 = pd.merge(df2, cc1, on='date_range_start')
df4 = pd.merge(df3, noaa, on = 'date_range_start')

In [35]:
df4.tail()

,id,dayofmonth,poi_cbg,region,naics_code,safegraph_place_id,date_range_start,city,bizgroup,dailyvisits,date_range_end,dayofweek,dayname,weekend,tot_pop,tot_male_pop,median_age_tot_pop,median_age_male,median_age_female,tot_white_pop,tot_blk_pop,tot_Asian_pop,tot_num_house-holds,num_of_family_HH,num_of_fam_HH_with_marr_couples,EDU_under,EDU_higher,median_HH_income,aggregate_HH_income,per_capita_income,FIPS_5digits,county,state,fips,cases,deaths,STATION,NAME,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS
8819,1981319,26,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,29,2020-10-27,1,Tue,0,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.0,0.391204,41648.0,16304800.0,25449.0,56021,Laramie,Wyoming,56021,725,4.0,USC00481676,"CHEYENNE WEATHER FORECAST OFFICE, WY US",0.0,0.0,0.0,66,32,48
8820,1981319,27,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,29,2020-10-28,2,Wed,0,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.0,0.391204,41648.0,16304800.0,25449.0,56021,Laramie,Wyoming,56021,725,4.0,USC00481676,"CHEYENNE WEATHER FORECAST OFFICE, WY US",0.0,0.0,0.0,66,32,48
8821,1981319,28,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,21,2020-10-29,3,Thur,0,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.0,0.391204,41648.0,16304800.0,25449.0,56021,Laramie,Wyoming,56021,725,4.0,USC00481676,"CHEYENNE WEATHER FORECAST OFFICE, WY US",0.0,0.0,0.0,66,32,48
8822,1981319,29,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,21,2020-10-30,4,Fri,0,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.0,0.391204,41648.0,16304800.0,25449.0,56021,Laramie,Wyoming,56021,725,4.0,USC00481676,"CHEYENNE WEATHER FORECAST OFFICE, WY US",0.0,0.0,0.0,66,32,48
8823,1981319,30,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,6,2020-10-31,5,Sat,1,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.0,0.391204,41648.0,16304800.0,25449.0,56021,Laramie,Wyoming,56021,725,4.0,USC00481676,"CHEYENNE WEATHER FORECAST OFFICE, WY US",0.0,0.0,0.0,66,32,48


In [36]:
df4[df4[['cases', 'deaths']].isna().any(axis=1)]['date_range_start'].nunique()

0

It looks like my city does not have any missing values in the dily count of COVID cases and COVID deaths.

In [37]:
df4['cases'].describe()

count    8824.000000
mean      322.779352
std       280.305247
min        35.000000
25%        35.000000
50%       270.000000
75%       725.000000
max       725.000000
Name: cases, dtype: float64

In [38]:
df4['deaths'].describe()

count    8824.000000
mean        1.890073
std         1.611522
min         0.000000
25%         0.000000
50%         2.000000
75%         4.000000
max         4.000000
Name: deaths, dtype: float64

In [39]:
df4[df4[['PRCP', 'TOBS']].isna().any(axis=1)]['date_range_start'].nunique()

0

The data does not have any missing values for precipitation and air temperature.

In [40]:
df4 = df4[~df4.isna().any(axis=1)].copy()

Question 2a stated, how many dates in your “LocalMerged” have missing values in the daily count of COVID cases and COVID deaths? 0 dates have missing values

Since 0 dates are missing, there is no need to impute missing values for COVID cases equal to 0 and COVID deaths equal to 0. (NY Times only report COVID information after a COVID case started to occur in a local county.) After the imputation, what are the average daily COVID cases and daily COVID deaths in your “LocalMerged”?

Question 2b (2 points): Use online search engines to find COVID related policies for your city or state. Pay special attention to type and timing of policies (e.g. shutdown and different phases of reopening). Create one or multiple variables to describe these policies. For example, if your city/state was shutdown at 2020-03-15, partially reopened on 2020-4-15, and full reopened on 2020-6-25, you can define Shutdown equal to one if the date is between 2020-03-15 and 2020-04-14, zero otherwise; Reopen1 equal to one if the date is between 2020-04-15 and 2020-06-24, zero otherwise; and Reopen2 equal to one if the date is on or after 2020-06-25.

 

How many policy variables do you create? What is their specific definition? Justify why you choose them this way.

In [41]:
df2['date_range_start'] = pd.to_datetime(df2['date_range_start'])

In [42]:
df2.tail()

,id,dayofmonth,poi_cbg,region,naics_code,safegraph_place_id,date_range_start,city,bizgroup,dailyvisits,date_range_end,dayofweek,dayname,weekend,tot_pop,tot_male_pop,median_age_tot_pop,median_age_male,median_age_female,tot_white_pop,tot_blk_pop,tot_Asian_pop,tot_num_house-holds,num_of_family_HH,num_of_fam_HH_with_marr_couples,EDU_under,EDU_higher,median_HH_income,aggregate_HH_income,per_capita_income,FIPS_5digits
12812,1981319,26,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,29,2020-10-27,1,Tue,0,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.0,0.391204,41648.0,16304800.0,25449.0,56021
12813,1981319,27,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,29,2020-10-28,2,Wed,0,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.0,0.391204,41648.0,16304800.0,25449.0,56021
12814,1981319,28,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,21,2020-10-29,3,Thur,0,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.0,0.391204,41648.0,16304800.0,25449.0,56021
12815,1981319,29,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,21,2020-10-30,4,Fri,0,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.0,0.391204,41648.0,16304800.0,25449.0,56021
12816,1981319,30,560210010004,WY,445120,sg:5a08cd5224084de29a8a5a52cc8f46f4,2020-10-01,Cheyenne,3.0,6,2020-10-31,5,Sat,1,635,286,40.1,39.9,40.3,545,69,0,337,190,127,0.0,0.391204,41648.0,16304800.0,25449.0,56021


In [43]:
# This code should insert number 1 for when the covid restriction where done
# and 0 otherwise
#df4['health'] = np.where(df['date_range_start'].isin(pd.to_datetime(['2020-04-01', '2020-10-01']) 1, 0))

I created one policy variable that is health order that requires mask wearing. it was in place from March 19th, 2020 until Feb 25th, 2021.

* The answer to question 2b, how many policy variables do you create is one.
* The answer to the question what is their specific definition? 1 = COVID restriction and 0 = otherwise. I choose this since it was the longest health restriction in Cheyenne.

Question 2c (2 points): Now define your dependent variable as Log(DailyVisits+1). Define the list of your potential right-hand variables, which should include some industry indicators (generated from NAICS codes), day of week indicators (Monday, Tuesday …, Sunday), some demographics, some weather variables, some variables for COVID cases and deaths, and at least one variable describing COVID-related policies. Feel free to add new variables to this list, as long as you articulate a reason for their relevance.

 

Summarize all these variables in a table with the number of observations, mean, median, standard deviation, minimum and maximum. If some variables are categorical and thus cannot appear in this summary table, create separate table(s) to describe the frequency of each value in such categorical variable(s).

 

In [44]:
X = df4.loc[:,['dayofweek','weekend', 'bizgroup', 'tot_pop', 'tot_male_pop', 
                'median_age_tot_pop', 'median_age_male', 'median_age_female', 'tot_white_pop',
                'tot_blk_pop', 'tot_Asian_pop', 'tot_num_house-holds', 'num_of_family_HH',
                'num_of_fam_HH_with_marr_couples', 'EDU_under', 'EDU_higher', 'median_HH_income', 
                'aggregate_HH_income', 'per_capita_income', 'PRCP', 'TOBS', 'cases', 'deaths']]
y = np.log(df4.loc[:, 'dailyvisits'] + 1)

In [45]:
# Converting categorical variables to strings
X['dayofweek']=X['dayofweek'].astype('str')
X['weekend']=X['weekend'].astype('str')
X['bizgroup']=X['bizgroup'].astype('str')

For Ridge or Lasso, all the X variables are all numerical. So I check below.

In [46]:
X.describe(include=[object])

,dayofweek,weekend,bizgroup
count,8824,8824,8824
unique,7,2,3
top,3,0,1.0
freq,1440,6435,7139


In [47]:
# get dummies for all categorical variables
X = pd.get_dummies(data = X, drop_first = True)
X.head()

,tot_pop,tot_male_pop,median_age_tot_pop,median_age_male,median_age_female,tot_white_pop,tot_blk_pop,tot_Asian_pop,tot_num_house-holds,num_of_family_HH,num_of_fam_HH_with_marr_couples,EDU_under,EDU_higher,median_HH_income,aggregate_HH_income,per_capita_income,PRCP,TOBS,cases,deaths,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,weekend_1,bizgroup_2.0,bizgroup_3.0
0,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,0.0,23,35,0.0,0,1,0,0,0,0,0,0,0
1,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,0.0,23,35,0.0,0,1,0,0,0,0,0,0,0
2,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,0.0,23,35,0.0,0,1,0,0,0,0,0,0,0
3,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,0.0,23,35,0.0,0,1,0,0,0,0,0,0,0
4,1819,826,37.3,35.1,43.8,1675,12,83,688,335,279,0.075806,0.504032,59808.0,43550200.0,26696.0,0.0,23,35,0.0,0,1,0,0,0,0,0,0,0


In [48]:
X.dtypes

tot_pop                              int64
tot_male_pop                         int64
median_age_tot_pop                 float64
median_age_male                    float64
median_age_female                  float64
tot_white_pop                        int64
tot_blk_pop                          int64
tot_Asian_pop                        int64
tot_num_house-holds                  int64
num_of_family_HH                     int64
num_of_fam_HH_with_marr_couples      int64
EDU_under                          float64
EDU_higher                         float64
median_HH_income                   float64
aggregate_HH_income                float64
per_capita_income                  float64
PRCP                               float64
TOBS                                 int64
cases                                int64
deaths                             float64
dayofweek_1                          uint8
dayofweek_2                          uint8
dayofweek_3                          uint8
dayofweek_4

In [49]:
X.describe(exclude=[object])

,tot_pop,tot_male_pop,median_age_tot_pop,median_age_male,median_age_female,tot_white_pop,tot_blk_pop,tot_Asian_pop,tot_num_house-holds,num_of_family_HH,num_of_fam_HH_with_marr_couples,EDU_under,EDU_higher,median_HH_income,aggregate_HH_income,per_capita_income,PRCP,TOBS,cases,deaths,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,weekend_1,bizgroup_2.0,bizgroup_3.0
count,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8.824000e+03,8824.000000,8824.0,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000,8824.000000
mean,1642.462149,841.510766,40.919016,39.158160,43.823719,1455.685857,26.880100,27.284905,664.027425,363.573549,277.094175,0.108733,0.368438,46819.285131,3.911598e+07,25967.380326,0.0,44.725295,322.779352,1.890073,0.130553,0.153558,0.163191,0.151405,0.140186,0.130553,0.270739,0.017339,0.173617
std,603.719433,290.064353,7.789664,7.261023,9.300253,597.381124,30.204515,31.192179,202.942421,185.083552,149.778859,0.072460,0.116857,22858.899516,2.099795e+07,5745.874930,0.0,17.301318,280.305247,1.611522,0.336930,0.360545,0.369561,0.358464,0.347199,0.336930,0.444367,0.130539,0.378802
min,503.000000,232.000000,19.900000,24.300000,19.000000,455.000000,0.000000,0.000000,250.000000,122.000000,49.000000,0.000000,0.158568,18826.000000,1.152410e+07,10981.000000,0.0,23.000000,35.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1480.000000,795.000000,34.700000,35.100000,35.100000,1213.000000,5.000000,0.000000,604.000000,282.000000,223.000000,0.069767,0.258478,18826.000000,2.548130e+07,20808.000000,0.0,23.000000,35.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1480.000000,826.000000,37.300000,38.800000,43.800000,1213.000000,13.000000,18.000000,677.000000,335.000000,246.000000,0.075806,0.365931,53214.000000,3.179480e+07,26432.000000,0.0,48.000000,270.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1819.000000,899.000000,49.600000,45.300000,53.800000,1675.000000,37.000000,42.000000,688.000000,356.000000,279.000000,0.203466,0.500000,59808.000000,4.355020e+07,27467.000000,0.0,64.000000,725.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,3692.000000,1840.000000,56.400000,60.600000,57.000000,3547.000000,158.000000,83.000000,1318.000000,1097.000000,884.000000,0.236842,0.689956,120096.000000,1.151429e+08,47560.000000,0.0,64.000000,725.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


The answer to question 2c is the tables are above that summarize all these variables with the number of observations, mean, median, standard deviation, minimum and maximum. A table is also above and shows variables that are categorical and thus cannot appear in this summary table and the frequency of each value in such categorical variable(s) is described.

# Part 3: Linear Regression, LASSO & Ridge

* The package used is called `scikit-learn`, also named `sklearn` in python. This is the most common package in ML in python (at least for these regular ML algorithms).

Question 3a (2 points): Choose a list of variables to predict log(dailyvisits+1). The list you choose could be a subset of the potential right-hand variables you have identified in Question 2c. I will refer to these variables as “features”.

 

Use the Python command “train_test_split()” to separate your “LocalMerged” into a training sample and a test sample.

 

Use function MinMaxScalar() to standardize all the “features” you choose, in both the training and test samples.

 

How many observations are there in the training sample? How many in the test sample? Choose at least two variables to comment on the difference between the two samples. For example, are they different in the months covered? Are they different in weather? Feel free to use other variables for this comparison.

The first thing I will do is to split our data into training and testing data. The train_test_split() function does this for us. By default, if we do not specify train_size or test_size, the function will split the data randomly, with $\approx$ 75% of the data in the train data, and $\approx$ 25% in the test data.

Remember to specify `random_state = `, so that you don't get a different split every time to run this line.

In [50]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state = 1)

In [51]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train

,tot_pop,tot_male_pop,median_age_tot_pop,median_age_male,median_age_female,tot_white_pop,tot_blk_pop,tot_Asian_pop,tot_num_house-holds,num_of_family_HH,num_of_fam_HH_with_marr_couples,EDU_under,EDU_higher,median_HH_income,aggregate_HH_income,per_capita_income,PRCP,TOBS,cases,deaths,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,weekend_1,bizgroup_2.0,bizgroup_3.0
5649,1373,754,30.1,27.3,40.6,753,36,64,525,257,49,0.096921,0.244014,31353.0,19251200.0,16422.0,0.0,64,270,2.0,0,0,0,0,0,1,1,0,0
2567,1425,730,42.2,49.4,28.4,1401,0,0,591,329,213,0.020309,0.245648,51913.0,33412100.0,24948.0,0.0,23,35,0.0,0,0,0,0,1,0,1,0,0
5643,1373,754,30.1,27.3,40.6,753,36,64,525,257,49,0.096921,0.244014,31353.0,19251200.0,16422.0,0.0,64,270,2.0,0,0,0,0,0,0,0,0,0
7563,2010,1023,31.0,26.1,45.4,1831,80,0,774,435,305,0.135647,0.365931,53214.0,48790000.0,26432.0,0.0,48,725,4.0,0,1,0,0,0,0,0,0,1
4490,1480,899,49.6,45.3,53.8,1213,37,18,677,282,223,0.203466,0.258478,18826.0,25481300.0,20808.0,0.0,64,270,2.0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,1293,699,34.7,34.2,35.5,1227,8,0,399,356,313,0.000000,0.563802,111875.0,44232500.0,35138.0,0.0,23,35,0.0,0,0,1,0,0,0,0,0,0
7813,1602,869,35.7,33.4,36.4,1319,41,42,659,393,295,0.040316,0.300613,63681.0,46050900.0,28792.0,0.0,48,725,4.0,0,0,1,0,0,0,0,0,1
905,1480,899,49.6,45.3,53.8,1213,37,18,677,282,223,0.203466,0.258478,18826.0,25481300.0,20808.0,0.0,23,35,0.0,0,0,0,0,1,0,1,0,1
5192,1434,716,36.1,40.2,31.6,1337,0,9,613,340,234,0.084314,0.437255,44077.0,33406400.0,23979.0,0.0,64,270,2.0,0,0,0,0,0,0,0,0,0


In [52]:
len(X_train_scaled)

6618

In [53]:
len(X_test_scaled)

2206

In [54]:
len(y_train)

6618

In [55]:
len(y_test)

2206

In [56]:
X_train['median_HH_income'].describe()

count      6618.000000
mean      46781.721366
std       22822.409820
min       18826.000000
25%       18826.000000
50%       53214.000000
75%       59808.000000
max      120096.000000
Name: median_HH_income, dtype: float64

In [57]:
X_train['per_capita_income'].describe()

count     6618.000000
mean     25969.930946
std       5743.285132
min      10981.000000
25%      20808.000000
50%      26432.000000
75%      27467.000000
max      47560.000000
Name: per_capita_income, dtype: float64

Question 3a asked, and the answer to how many observations are there in the training sample is 6618. The answer to how many in the test sample is 2206. Two variables to comment on is median_HH_income and per_capita. The difference between the two samples is the average median income is much higher, almost more than twice, than per capita income. This means this data is random.

## Linear regression

Question 3b (2 points): Using Log(dailyvisits+1) and the standardized features in Question 3a, run a linear regression in your training sample. Report the coefficient estimates and R-squares.

 

What “features” have statistically significant coefficients (with p-value<0.05)? Among these significant “features”, which has a positive coefficient and which has a negative coefficient? Comment on whether their signs and magnitude confirm your prior.

 

If you extend the model to predict log(dailyvisits+1) in the test sample, what is the R-squares in the test sample?

In [58]:
import statsmodels.api as sm
X2_train = sm.add_constant(X_train)
results = sm.OLS(y_train,X2_train.astype(float)).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            dailyvisits   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                     52.87
Date:                Wed, 21 Apr 2021   Prob (F-statistic):          1.83e-247
Time:                        00:00:42   Log-Likelihood:                -9369.1
No. Observations:                6618   AIC:                         1.879e+04
Df Residuals:                    6591   BIC:                         1.898e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               0.0758      0.224      0.339      0.734      -0.362       0.514
tot_pop                             0.0004      0.000      1.007      0.314      -0.000       0.001
tot_male_pop                       -0.0015      0.000     -5.395      0.000      -0.002      -0.001
median_age_tot_pop                  0.0183      0.010      1.881      0.060      -0.001       0.037
median_age_male                    -0.0168      0.007     -2.507      0.012      -0.030      -0.004
median_age_female                   0.0113      0.005      2.200      0.028       0.001       0.021
tot_white_pop                      -0.0002      0.000     -0.809      0.419      -0.001       0.000
tot_blk_pop                        -0.0014      0.001     -2.074      0.038      -0.003   -7.62e-05
tot_Asian_pop                       0.0051      0.001      6.056      0.000       0.003       0.007
tot_num_house-holds                 0.0019      0.000      4.311      0.000       0.001       0.003
num_of_family_HH                    0.0038      0.000      8.548      0.000       0.003       0.005
num_of_fam_HH_with_marr_couples    -0.0004      0.001     -0.655      0.513      -0.002       0.001
EDU_under                          -4.9822      0.621     -8.028      0.000      -6.199      -3.766
EDU_higher                         -0.0076      0.305     -0.025      0.980      -0.605       0.590
median_HH_income                -1.205e-05   2.46e-06     -4.892      0.000   -1.69e-05   -7.22e-06
aggregate_HH_income             -2.667e-08   8.12e-09     -3.283      0.001   -4.26e-08   -1.07e-08
per_capita_income                2.809e-05   9.43e-06      2.979      0.003    9.61e-06    4.66e-05
PRCP                              6.17e-15   8.83e-16      6.990      0.000    4.44e-15     7.9e-15
TOBS                                0.0105      0.003      3.700      0.000       0.005       0.016
cases                               0.0008      0.001      1.169      0.242      -0.001       0.002
deaths                             -0.0475      0.141     -0.338      0.735      -0.323       0.228
dayofweek_1                         0.0365      0.048      0.766      0.444      -0.057       0.130
dayofweek_2                         0.1249      0.046      2.723      0.006       0.035       0.215
dayofweek_3                         0.1099      0.046      2.409      0.016       0.020       0.199
dayofweek_4                         0.2364      0.046      5.104      0.000       0.146       0.327
dayofweek_5                         0.1354      0.027      4.974      0.000       0.082       0.189
dayofweek_6                        -0.2042      0.028     -7.380      0.000      -0.258      -0.150
weekend_1                          -0.0688      0.028     -2.502      0.012      -0

### * A short digression: another way of running linear regression, using scikit-learn package.

In [59]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression().fit(X_train, y_train)

coef = linreg.coef_

print('R-squared score (training): {:.3f}'.format(linreg.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'.format(linreg.score(X_test, y_test)))

R-squared score (training): 0.173
R-squared score (test): 0.151


In [60]:
# the code below predicts y using X-test data
X2_test = sm.add_constant(X_test)
y_predict = results.predict(X2_test)
from sklearn.metrics import r2_score
# below R-sq for X2_test is coded
print(r2_score(y_test, y_predict))
# results1 = sm.OLS(y_test,X2_test.astype(float)).fit()
#results1.summary()

0.15086724590667078


Question 3b stated, using Log(dailyvisits+1) and the standardized features in Question 3a, run a linear regression in your training sample. Report the coefficient estimates and R-squares. The R-sq is 17.3 % of the variation in Y-train is explained by X-train.

 

What “features” have statistically significant coefficients (with p-value<0.05)? The features that have statistical significance are tot_male_pop, median_age_male, median_age_female, tot_blk_pop, tot_Asin_pop, tot

Question 3b stated, using Log(dailyvisits+1) and the standardized features in Question 3a, run a linear regression in your training sample. Report the coefficient estimates and R-squares. The R-sq is 17.3 % of the variation in Y-train is explained by X-train.

 

* What “features” have statistically significant coefficients (with p-value<0.05)? The features that have statistical significance are tot_male_pop, median_age_male, median_age_female, tot_blk_pop, tot_Asin_pop, tot_num_house_holds, num_of_family_HH, EDU_under, median_HH_income, aggregate_HH_income, per_capita_income, PRCP, TOBS, dayofweek_2, dayofweek_3, dayofweek_4, dayofweek_5, dayofweek_6, weekend_1, bizgroup_2.0, bizgroup_3.0.

* Among these significant “features”, which has a positive coefficient and which has a negative coefficient? Comment on whether their signs and magnitude confirm your prior.

 

* If you extend the model to predict log(dailyvisits+1) in the test sample, what is the R-squares in the test sample? The R-sq for the test is 15.1%.

In [61]:
# coef_sorted=sorted(coef, key = abs, reverse = True)
# for i, j in enumerate(coef_sorted):
#     print('{var} {coef:.5f}'.format(var = X.columns[i], coef = j))

Question 3c (2 points): Using Log(dailyvisits+1) and the standardized features in Question 3a, run LASSO in your training sample.

 

Try at least three different values for the penalty parameter. For each value you have tried, report the coefficient estimates and R-squares; also extend the model to predict log(dailyvisits+1) in the test sample, and report the R-squares in the test sample.

 

Among the penalty parameters you have tried, which one is most appropriate? Explain how you arrive at this choice.

 

In the LASSO model with your preferred choice of the penalty parameter, what “features” have zero (or very close to zero) coefficients? What and how many “features” have non-zero coefficients? Comment on whether their signs and magnitude confirm your prior. What is the R-squares when you apply this model to the test sample? Is it higher or lower than what you find in question 3b for linear regression?

 

Use the random permutation method to identify the five most important features in the above LASSO model. What are they? Comment on whether the order of their importance is consistent with your prior.

 

## Lasso

In [62]:
from sklearn.linear_model import Lasso

linlasso = Lasso(alpha=0.0001, max_iter = 10000).fit(X_train_scaled, y_train)

print('Non-zero features: {}'.format(np.sum(linlasso.coef_ != 0)))
print('R-squared score (training): {:.3f}'.format(linlasso.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}\n'.format(linlasso.score(X_test_scaled, y_test)))
print('Features with non-zero weight (sorted by absolute magnitude):')

for e in sorted (list(zip(list(X), linlasso.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))

Non-zero features: 26
R-squared score (training): 0.172
R-squared score (test): 0.150

Features with non-zero weight (sorted by absolute magnitude):
	num_of_family_HH, 3.651
	tot_male_pop, -1.974
	aggregate_HH_income, -1.958
	tot_num_house-holds, 1.812
	median_HH_income, -1.261
	EDU_under, -1.147
	bizgroup_2.0, -0.953
	per_capita_income, 0.648
	num_of_fam_HH_with_marr_couples, -0.562
	median_age_tot_pop, 0.561
	median_age_male, -0.526
	median_age_female, 0.443
	tot_Asian_pop, 0.419
	cases, 0.398
	TOBS, 0.389
	dayofweek_6, -0.275
	tot_blk_pop, -0.247
	dayofweek_4, 0.233
	tot_white_pop, -0.204
	dayofweek_2, 0.121
	dayofweek_3, 0.107
	bizgroup_3.0, 0.104
	EDU_higher, 0.092
	dayofweek_5, 0.062
	dayofweek_1, 0.033
	deaths, 0.015


In [63]:

linlasso = Lasso(alpha=0.01, max_iter = 10000).fit(X_train_scaled, y_train)

print('Non-zero features: {}'.format(np.sum(linlasso.coef_ != 0)))
print('R-squared score (training): {:.3f}'.format(linlasso.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}\n'.format(linlasso.score(X_test_scaled, y_test)))
print('Features with non-zero weight (sorted by absolute magnitude):')

for e in sorted (list(zip(list(X), linlasso.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))

Non-zero features: 11
R-squared score (training): 0.118
R-squared score (test): 0.097

Features with non-zero weight (sorted by absolute magnitude):
	deaths, 0.450
	bizgroup_2.0, -0.349
	TOBS, 0.268
	dayofweek_6, -0.265
	EDU_under, -0.220
	tot_Asian_pop, 0.173
	tot_blk_pop, -0.104
	dayofweek_4, 0.099
	bizgroup_3.0, 0.051
	EDU_higher, 0.046
	weekend_1, -0.004


In [64]:

linlasso = Lasso(alpha=0.05, max_iter = 10000).fit(X_train_scaled, y_train)

print('Non-zero features: {}'.format(np.sum(linlasso.coef_ != 0)))
print('R-squared score (training): {:.3f}'.format(linlasso.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}\n'.format(linlasso.score(X_test_scaled, y_test)))
print('Features with non-zero weight (sorted by absolute magnitude):')

for e in sorted (list(zip(list(X), linlasso.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))

Non-zero features: 2
R-squared score (training): 0.060
R-squared score (test): 0.050

Features with non-zero weight (sorted by absolute magnitude):
	deaths, 0.296
	TOBS, 0.147


We can compare the fitting results across different choices of alpha parameter:

In [65]:
for alpha in [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 5]:
    linlasso = Lasso(alpha=alpha, max_iter = 10000).fit(X_train_scaled, y_train)
    r2_train = linlasso.score(X_train_scaled, y_train)
    r2_test = linlasso.score(X_test_scaled, y_test)
    
    print('Alpha = {}\nFeatures kept: {}, r-squared training: {:.3f}, \
r-squared test: {:.3f}\n'
         .format(alpha, np.sum(linlasso.coef_ != 0), r2_train, r2_test))

Alpha = 1e-05
Features kept: 27, r-squared training: 0.173, r-squared test: 0.151

Alpha = 0.0001
Features kept: 26, r-squared training: 0.172, r-squared test: 0.150

Alpha = 0.001
Features kept: 21, r-squared training: 0.167, r-squared test: 0.145

Alpha = 0.01
Features kept: 11, r-squared training: 0.118, r-squared test: 0.097

Alpha = 0.1
Features kept: 1, r-squared training: 0.015, r-squared test: 0.013

Alpha = 1
Features kept: 0, r-squared training: 0.000, r-squared test: -0.001

Alpha = 5
Features kept: 0, r-squared training: 0.000, r-squared test: -0.001



In [66]:
from sklearn.linear_model import RidgeCV

n_alphas = 50
alphas = np.logspace(-5, 2, n_alphas)

linridge_cv = RidgeCV(alphas=alphas).fit(X_train_scaled, y_train)
linridge_cv.alpha_

0.2682695795279722

Question 3c

 
The 3 penalty parameter tried are .0001, .01, .05 respectively. The coefficient estimates are shown above and R-squares are .173, .118, .060, respectively; in the test sample, the R-squares in the test sample are .151, .097, .050, respectively.

 

Among the penalty parameters you have tried, which one is most appropriate? The alpha .0001 is the most appropriate since it has the biggest R-sq and test scores. I came to this conclusion since it is the model that has the highest variation of Y explained by X.

 

In the LASSO model with your preferred choice of the penalty parameter, what “features” have zero (or very close to zero) coefficients? The penalty parametrs that have zero or very close zero coefficients are above. What and how many “features” have non-zero coefficients? the features with non zero coefficients are above and 26, 11, 2 have non-zero coefficient respectively. Comment on whether their signs and magnitude confirm your prior.Their signs and magnitude did not confirm the prior.

What is the R-squares when you apply this model to the test sample? The R-sq is lower than what I found in 3b.

 

Use the random permutation method to identify the five most important features in the above LASSO model. What are they? .00001, .0001, .001, .01, .1 are the five most important features respectively. The R-squares are .173, .172, .167, .118, .015 respectively;Yes the order of their importance is consistent with your prior.

Question 3d (2 points): Using Log(dailyvisits+1) and the standardized features in Question 3a, run Ridge in your training sample.

 

Try at least three different values for the penalty parameter. For each value you have tried, report the coefficient estimates and R-squares; also extend the model to predict log(dailyvisits+1) in the test sample, and report the R-squares in the test sample.

 

Among the penalty parameters you have tried, which one is most appropriate? Explain how you arrive at this choice.

 

In the Ridge model with your preferred choice of the penalty parameter, what “features” have zero (or very close to zero) coefficients? What and how many “features” have non-zero coefficients? Comment on whether their signs and magnitude confirm your prior. What is the R-squares when you apply this model to the test sample? Is it higher or lower than what you find in question 3b for linear regression and question 3c for LASSO?

 

Use the random permutation method to identify the five most important features in the above Ridge model. What are they? Comment on whether the order of their importance is consistent with your prior. Does Ridge identify the same five most important features as LASSO?

## Ridge regression

In [67]:
# fit the ridge linear regression model
from sklearn.linear_model import Ridge
linridge = Ridge(alpha=0.1).fit(X_train_scaled, y_train)

print('Ridge regression coef:\n{}'.format(linridge.coef_))
print('R-squared score (training): {:.3f}'.format(linridge.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}'.format(linridge.score(X_test_scaled, y_test)))
print('Number of non-zero features: {}'.format(np.sum(linridge.coef_ != 0)))

Ridge regression coef:
[ 0.98140877 -2.32789681  0.64286192 -0.58031657  0.43237955 -0.61592207
 -0.22980762  0.41440972  1.92241289  3.63253628 -0.41398856 -1.16654373
  0.03029322 -1.2381804  -2.45036731  0.90712963  0.          0.34242731
  0.19821301  0.24279304  0.03653715  0.12467859  0.10986836  0.23628243
  0.13532036 -0.20426407 -0.06894371 -0.93660394  0.09709967]
R-squared score (training): 0.173
R-squared score (test): 0.151
Number of non-zero features: 28


In [68]:
linridge2 = Ridge(alpha=0.5).fit(X_train_scaled, y_train)

print('Ridge regression coef:\n{}'.format(linridge.coef_))
print('R-squared score (training): {:.3f}'.format(linridge.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}'.format(linridge.score(X_test_scaled, y_test)))
print('Number of non-zero features: {}'.format(np.sum(linridge.coef_ != 0)))

Ridge regression coef:
[ 0.98140877 -2.32789681  0.64286192 -0.58031657  0.43237955 -0.61592207
 -0.22980762  0.41440972  1.92241289  3.63253628 -0.41398856 -1.16654373
  0.03029322 -1.2381804  -2.45036731  0.90712963  0.          0.34242731
  0.19821301  0.24279304  0.03653715  0.12467859  0.10986836  0.23628243
  0.13532036 -0.20426407 -0.06894371 -0.93660394  0.09709967]
R-squared score (training): 0.173
R-squared score (test): 0.151
Number of non-zero features: 28


In [69]:
linridge3 = Ridge(alpha=0.01).fit(X_train_scaled, y_train)

print('Ridge regression coef:\n{}'.format(linridge.coef_))
print('R-squared score (training): {:.3f}'.format(linridge.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}'.format(linridge.score(X_test_scaled, y_test)))
print('Number of non-zero features: {}'.format(np.sum(linridge.coef_ != 0)))

Ridge regression coef:
[ 0.98140877 -2.32789681  0.64286192 -0.58031657  0.43237955 -0.61592207
 -0.22980762  0.41440972  1.92241289  3.63253628 -0.41398856 -1.16654373
  0.03029322 -1.2381804  -2.45036731  0.90712963  0.          0.34242731
  0.19821301  0.24279304  0.03653715  0.12467859  0.10986836  0.23628243
  0.13532036 -0.20426407 -0.06894371 -0.93660394  0.09709967]
R-squared score (training): 0.173
R-squared score (test): 0.151
Number of non-zero features: 28


Now, we also want to know which variables are the most important ones in our model.

In [70]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(linridge).fit(X_test_scaled, y_test)
eli5.show_weights(perm)

Weight,Feature
0.8163 ± 0.0750,x9
0.4319 ± 0.0333,x14
0.3209 ± 0.0386,x1
0.2180 ± 0.0180,x8
0.2042 ± 0.0311,x11
0.1431 ± 0.0217,x13
0.0530 ± 0.0088,x0
0.0351 ± 0.0086,x15
0.0344 ± 0.0103,x17
0.0326 ± 0.0104,x7


# Question 3d

The coefficient estimates and R-squares for each penalty (linridge, linridge2, linridge3) are above, respectively. Alpha = .1, .05, and .01 were all the most appropriate parameter since it had the highest R-squared from the training and test scores. The high R-squares mean it explained the most variation in y given X.

The “features” that have zero (or very close to zero) coefficients are shown above. The many “features” that have non-zero coefficients are shown above. Their signs and magnitude confirm the prior. The R-squares when I apply this model to the test sample is above. It is lower than what I find in question 3b for linear regression and question 3c for LASSO?

 

Use the random permutation method to identify the five most important features in the above Ridge model. They are above. The order of their importance is consistent with my prior. Does Ridge identify the same five most important features as LASSO? yes it does.